In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")


indices<-Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from

In [2]:
auto_index <- indices$NseTimeSeries() %>%
                 filter(NAME == "NIFTY 50") %>%  
                 arrange(desc(TIME_STAMP)) %>%
                 print(n=5)

# Source:     lazy query [?? x 7]
# Database:   Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Ordered by: desc(TIME_STAMP)
  NAME     TIME_STAMP   HIGH    LOW   OPEN  CLOSE    VOLUME
  <chr>    <date>      <dbl>  <dbl>  <dbl>  <dbl>     <dbl>
1 NIFTY 50 2019-08-30 11043. 10875. 10988. 11023. 628154431
2 NIFTY 50 2019-08-29 11021. 10922. 10996. 10948. 649876160
3 NIFTY 50 2019-08-28 11130. 10988. 11101. 11046. 549954696
4 NIFTY 50 2019-08-27 11142. 11050. 11107. 11105. 685551267
# … with more rows


In [3]:
auto <- as.data.frame(auto_index)

In [4]:
returns <- (c(diff(auto$CLOSE,lag=1)*(-1),0)) 

In [5]:
length(returns)

[1]   74.95  -97.80  -59.25   47.50  228.50   88.00 -177.35  -98.30  -36.90
  [10]    6.10   18.40  103.55 -183.80   77.20  176.95  -92.75   85.65 -134.75
  [19]   17.35 -138.00   32.60 -103.80  -95.10   32.15  -19.15  -59.75  -15.15
  [28]  -73.05 -177.65  -90.60   24.90   74.25   35.85  -30.40   84.00  -57.00
  [37]   -2.70 -252.55 -135.60   30.00    6.45   44.70   76.75  -52.70   -6.00
  [46]   51.10   96.80  -24.45 -107.65  140.30   -0.05   19.35 -151.15  -90.75
  [55]    7.85  -59.40   42.90   52.05   26.90 -177.90  -66.90  165.75  -23.10
  [64]   84.80  -67.65    4.00   80.65  187.05  -80.85   28.80 -119.15  421.10
  [73]  150.05  100.10  -65.05   73.85 -130.70  -22.90  -57.65 -138.45 -100.35
  [82] -114.00  -12.50  -23.40   -6.50  112.85  -84.35  150.20  -18.50 -158.35
  [91]  -34.35   96.80   46.90   46.75   12.40  -87.65   67.45  -61.45   67.95
 [100]  -45.95  -69.25   44.05   45.25   53.90  124.95  -38.20  129.00 -102.65
 [109]  -64.15  -11.35   70.20   35.35   83.60    1.55   40.50  133.15  132.65
 [118]  -22.80    5.20   65.55  123.95   71.00  -14.15  -28.65  -44.80   88.45
 [127]    1.80   54.40  131.10  -36.60  -83.45  -21.65  -47.60  -37.75  -57.40
 [136]  -54.80 -125.80    6.95  128.10   22.10   18.60   62.70  179.15   -0.40
 [145]   -9.35 -119.00  -69.25   18.30  -91.25  -39.10   54.90    1.75   14.90
 [154]    3.50  149.20  -57.35  -26.65  -33.55   53.00   30.35   44.45   55.10
 [163] -120.25 -117.60   47.55    2.65   80.10   49.95   66.35  -90.50 -197.70
 [172]  -15.60   58.60   20.35   82.90   13.90   53.95  188.45   60.70 -205.25
 [181]   92.55 -181.75  -86.60  -14.25    7.00   18.05  129.85   43.25   57.00
 [190]  101.85  -73.30  -56.15 -107.20   81.20   65.50   40.40   -6.20  100.30
 [199] -103.00  -13.20   68.40    6.00  -29.00  172.55   -6.15  188.20  -52.45
 [208]  220.85  -94.90  -99.85   77.95  -98.45  -58.30 -149.50 -131.70   72.25
 [217]   40.00  237.85 -225.45  159.05  -47.00   31.60 -282.80 -259.00 -150.05
 [226]   77.85  -47.10  -76.25  -13.65  100.05 -175.70  -91.25  -44.55  -98.85
 [235] -137.45  145.30   82.40 -150.60 -151.00   52.20   59.95  -43.35  -62.05
 [244]  -98.15    3.70  -15.10  -46.60   46.55  134.85  -25.65   11.85   19.15
 [253]   81.00   85.70  -50.05   79.35  -73.75  -41.20   20.70   60.55    2.35
 [262]   26.30  116.10 -101.50  -10.30   36.95   41.20  111.05   35.30   -2.30
 [271]   49.55   74.55   53.10  -23.35  -27.60   71.20  -82.05   -4.30   74.90
 [280]    1.05   94.35   80.25   22.90  -20.15   70.00   42.60  -57.00  125.20
 [289]  -82.30  -97.75    6.70  -59.40   80.75  -30.95   61.60  -89.40  -17.85
 [298]    9.65  -48.65   13.85   55.90   19.30   -0.70   83.70   91.50  -35.35
 [307]  -67.70  -39.95  121.80  -18.95  -55.35   83.50   91.30   83.50 -106.35
 [316]   20.00  -79.70  -86.30  -58.40  -60.75   -4.75    0.10   89.95  -25.15
 [325]   23.90    2.30   97.25  -61.40  -38.40  -21.30   47.05   74.50   47.25
 [334]  -43.80   29.65   20.65   -1.25   39.10  -22.50   20.35   47.75   21.95
 [343]   41.50   14.90   22.90   47.75    6.45  196.75 -116.60   33.20   98.10
 [352]  -70.45   53.50  132.60 -116.70  -40.50   30.90   30.10 -100.90 -165.00
 [361]  -50.75  -15.95    5.45  194.55  -15.80   88.45  -95.05 -109.60  -99.50
 [370]  -34.50  -61.45  -28.30   91.55  108.35  -14.75   37.05  -18.00  -73.90
 [379]  -93.20   44.60  -38.85   84.80 -121.90  100.15  -21.55 -168.30  -94.05
 [388] -256.30  -10.80  -21.95  -80.75   60.75  -16.35    2.30  117.50   71.50
 [397]   77.70   28.45   88.10  -41.10   60.30   30.05   19.00   -4.80   13.40
 [406]   64.75   54.05   61.60    1.00    6.65  -95.15   52.80  -12.85  -40.75
 [415]   38.50   52.70   -3.90  -19.00   74.45   55.50   81.15   59.15  -47.20
 [424]  -82.10   56.60   98.95  122.60  -74.15   -9.50    5.95 -104.75 -134.75
 [433]   -8.95  -29.30    9.85   40.95    6.45   15.40   28.15   15.15   68.85
 [442]   96.70  -68.55  -38.35  -96.80   12.80    5.80  -47.00 -101.65   -0.70
 [451]   28.70  -16.70  105.20  -28.35   40.60  -20.7

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)